# Ocean Flow Data Analysis Project

## Imports, Settings, Function Definitions, and Loading Initial Data

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import math
import random
import seaborn as sns
import scipy.sparse as sp
from tqdm import tqdm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from scipy.stats import norm
from sklearn.model_selection import KFold
import torch.multiprocessing as mp
from itertools import product
# Suppress only the ConvergenceWarning
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print("Using device:", device)
torch.set_default_device(device=device)
torch.set_default_dtype(torch.float64)

from PIL import Image
import re
import os

# Global Variables
DATA_DIR = "OceanFlow/"
NUM_TIMESTEPS = 100 # T = 1 to 100
GRID_SPACING_KM = 3
TIME_SPACING_H = 3
PAUSE_BETWEEN_FRAMES = 100 # milliseconds

# Import custom functions
from ocean_flow_utility import (
    squared_exponential_kernel, 
    euclidean_distance, 
    select_distant_indices, 
    compute_magnitude, 
    load_flow_data, 
    plot_vector_field, 
    plot_single_frame, 
    compute_conditional_mean_variance_fixed_hyperparams, 
    causal_moving_average,
    plot_particle_coordinates_traces_and_trajectories_for_timestep, 
    plot_particle_debris_movement, 
    kernel_multi_dimension, 
    create_partitions, 
    compute_log_likelihood, 
    logpdf_normal, 
    simulate_expanded_particle_movement, 
    plot_particle_coordinates_traces_and_trajectories_for_timestep_days, 
    plot_particle_coordinates_traces_and_trajectories_for_timestep_days_monitoring_stations
)

from ocean_flow_parallel import (
    compute_log_likelihood, 
    rbf_kernel, 
    compute_conditional, 
    compute_ll_for_var_lengthscale_of_U_and_V_component, 
    predict_conditional_mean_variance_fixed_hyperparams, 
    find_future_magnitude_given_coordinates_in_kilometers, 
    simulate_expanded_particle_debris_scattering, 
    simulate_expanded_imputed_particle_debris_scattering, 
    remove_land_coordinates_and_beach_debris_X_imputed_y_imputed
)

import em
import common
import imageio


# from ocean_flow_utility import *

# Load the first file to determine shape
sample_u = np.loadtxt(os.path.join(DATA_DIR, "1u.csv"), delimiter=",")
ny, nx = sample_u.shape

# Initialize 3D arrays
U = np.zeros((NUM_TIMESTEPS, ny, nx))
V = np.zeros((NUM_TIMESTEPS, ny, nx))

# Load each timestep into the 3D array
for t in range(1, NUM_TIMESTEPS + 1):
    u_path = os.path.join(DATA_DIR, f"{t}u.csv")
    v_path = os.path.join(DATA_DIR, f"{t}v.csv")
    U[t-1] = np.loadtxt(u_path, delimiter=",")
    V[t-1] = np.loadtxt(v_path, delimiter=",")

U = torch.from_numpy(U).to(device)
V = torch.from_numpy(V).to(device)
U = U.to(device)
V = V.to(device)
MAG = (U**2 + V**2)**0.5

True
NVIDIA GeForce RTX 2080 Ti


## HW Problems

In [ ]:
# About the data
#
# Time:
# These are flow vectors for time T 1 to 100.
# Thus, for example, 1u.csv gives data at a time coordinate of 0 hours. 
#
# Depth:
# This is the average flow from a depth near the surface to the shallower of either the bottom or 400m.
# 
# U component:
# *u.csv contains the horizontal components of the vectors.
#
# V Component: 
# *v.csv contains the vertical components of the vectors.
#
# Spacing:
# The grid spacing is 3 km.
# The matrix index (0, 0) is the bottom left of the plot.
#
# X-axis:
# The columns of the u & v component csv represent the x-axis & correspond to the horizontal direction.
#
# Y-axis:
# The rows of the u & v component csv represent the y-axis & correspond to the vertical direction.
#
# The magnitude is represented by the square-root of the squares of the sum of the squares of the u and v components.


### Loading the data into a 3D array format

In [ ]:
# To load the data into a 3D array format, you want to construct two arrays:
#     U[t, y, x] for horizontal flow components over time
#     V[t, y, x] for vertical flow components over time
# This means:
#     The first axis = time steps (from 1 to 100)
#     The second axis = y direction (rows in the CSV)
#     The third axis = x direction (columns in the CSV)

In [ ]:
# # Load the first file to determine shape
# sample_u = np.loadtxt(os.path.join(DATA_DIR, "1u.csv"), delimiter=",")
# ny, nx = sample_u.shape

# # Initialize 3D arrays
# U = np.zeros((NUM_TIMESTEPS, ny, nx))
# V = np.zeros((NUM_TIMESTEPS, ny, nx))

# # Load each timestep into the 3D array
# for t in range(1, NUM_TIMESTEPS + 1):
#     u_path = os.path.join(DATA_DIR, f"{t}u.csv")
#     v_path = os.path.join(DATA_DIR, f"{t}v.csv")
#     U[t-1] = np.loadtxt(u_path, delimiter=",")
#     V[t-1] = np.loadtxt(v_path, delimiter=",")

# MAG = np.sqrt(U**2 + V**2)

In [ ]:
# MAG = np.sqrt(U**2 + V**2)

In [ ]:
# MAG_VAR = np.var(MAG, axis=0)

In [ ]:
# min_var = np.min(MAG_VAR[MAG_VAR != 0])


In [ ]:
# y, x = np.where(MAG_VAR == min_var)

In [ ]:
# y[0] * 3

In [ ]:
# x[0] * 3

### Maximum x-velocity

In [ ]:
max_velocity = np.max(U)
max_index = np.unravel_index(np.argmax(U), U.shape)
t_max, y_max, x_max = max_index

print(f"Max x-axis velocity: {max_velocity}")
print(f"Location: hour={(t_max)*3}, y km={y_max*3}, x km ={x_max*3}")

### Problem 1.c

In [ ]:
avg_velocity = np.mean(MAG)

In [ ]:
avg_velocity

In [ ]:
np.mean(U)

In [ ]:
np.mean(V)

### Identifying-water Only Locations & Correlations of long-range distances

In [ ]:
# Load the first file to determine shape
sample_u = np.loadtxt(os.path.join(DATA_DIR, "1u.csv"), delimiter=",")
ny, nx = sample_u.shape

# Initialize 3D arrays
U = np.zeros((NUM_TIMESTEPS, ny, nx))
V = np.zeros((NUM_TIMESTEPS, ny, nx))

# Load each timestep into the 3D array
for t in range(1, NUM_TIMESTEPS + 1):
    u_path = os.path.join(DATA_DIR, f"{t}u.csv")
    v_path = os.path.join(DATA_DIR, f"{t}v.csv")
    U[t-1] = np.loadtxt(u_path, delimiter=",")
    V[t-1] = np.loadtxt(v_path, delimiter=",")

mask = pd.read_csv('OceanFlow/mask.csv', header=None).values  # Shape: (504, 555)
# The mask is upside-down
mask = np.flipud(mask)

INDEX = 0
plt.imshow(U[INDEX, :, :],origin="lower", cmap="viridis", extent=[0, U[INDEX, :, :].shape[1], 0, U[INDEX, :, :].shape[0]], aspect='equal')
# plt.imshow(mask_flipped, origin="lower", cmap="viridis", extent=[0, mask_flipped.shape[1], 0, mask_flipped.shape[0]], aspect='equal')

# Plot the land over the water at time 0 

# Assume U and mask are already loaded
# U.shape = (100, 504, 555)
# mask.shape = (504, 555)

U_t_0 = U[0, :, :]  # time = 0

fig, ax = plt.subplots(figsize=(10, 9))

# Step 1: Display the U field
cax = ax.imshow(U_t_0,
                origin='lower',  # (0,0) bottom-left
                cmap='viridis',
                extent=[0, U_t_0.shape[1], 0, U_t_0.shape[0]],
                aspect='equal')

# Step 2: Overlay the mask
# Create a transparent mask: land is semi-transparent, water is fully transparent
mask_display = np.ma.masked_where(mask == 1, mask)  # mask out water, show land
ax.imshow(mask_display,
          origin='lower',
          cmap='gray',    # land will appear gray
          alpha=0.3,      # transparency (0=transparent, 1=opaque)
          extent=[0, U_t_0.shape[1], 0, U_t_0.shape[0]],
          aspect='equal')

# Step 3: Add colorbar and labels
fig.colorbar(cax, ax=ax, label='U[0, y, x] value')

ax.set_xlabel('x coordinate')
ax.set_ylabel('y coordinate')
ax.set_title('U[0, y, x] at Time = 0 with Land Mask Overlay')

plt.show()

# Plot the locations of the water on the map

# Assume U and mask are already loaded
# U.shape = (100, 504, 555)
# mask.shape = (504, 555)

# Step 1: Find water locations (mask == 1)
water_locations = np.argwhere(mask == 1)  # shape (n_water_points, 2)
y_coords, x_coords = water_locations[:, 0], water_locations[:, 1]

# Step 2: Plot U[0, :, :]
U_t_0 = U[0, :, :]

fig, ax = plt.subplots(figsize=(10, 9))

# Display the grid
cax = ax.imshow(U_t_0, 
                origin='lower',  # (0,0) at bottom-left
                cmap='viridis',  # colormap
                extent=[0, U_t_0.shape[1], 0, U_t_0.shape[0]],  # [x_min, x_max, y_min, y_max]
                aspect='equal')  # square pixels

# Add colorbar
fig.colorbar(cax, ax=ax, label='U[0, y, x] value')

# Step 3: Overlay water points
ax.scatter(x_coords, y_coords, 
           color='cyan', 
           s=1,  # small points
           label='Water points')

# Label axes
ax.set_xlabel('x coordinate')
ax.set_ylabel('y coordinate')
ax.set_title('U[0, y, x] at Time = 0 with Water Points')

# Optional: Add legend
ax.legend(loc='upper right')

plt.show()

# Assume U and mask are already loaded
# U.shape = (100, 504, 555)
# mask.shape = (504, 555)

# Step 1: Reshape U to (y*x, time)
U_reshape = U.transpose(1, 2, 0).reshape(504*555, 100)
V_reshape = V.transpose(1, 2, 0).reshape(504*555, 100)

# Step 2: Flatten the mask (y,x) -> (y*x,)
mask_flat = mask.flatten()  # shape (504*555,)

# Step 3: Keep only rows where mask == 1 (i.e., water)
water_indices = np.where(mask_flat == 1)[0]

U_water_data = U_reshape[water_indices]  # shape (n_water_points, 100)
V_water_data = V_reshape[water_indices]
# Step 4: Find corresponding (y, x) coordinates
y_coords = water_indices // 555
x_coords = water_indices % 555

# Step 5: Build the coordinate matrix
coords = list(zip(y_coords, x_coords))
# coords_matrix = np.stack([
#     np.tile(y_coords[:, np.newaxis], (1, 100)),
#     np.tile(x_coords[:, np.newaxis], (1, 100))
# ], axis=-1)  # shape (n_water_points, 100, 2)

print("Water-only data shape:", U_water_data.shape)
print("Coordinates matrix shape:", len(coords))

import numpy as np

# Step 6: Remove water points where U and V are all zeros across time
# U_water_data: (n_water_points, 100)

# Check where all values are exactly zero across all timesteps
nonzero_mask = ~(np.all(U_water_data == 0, axis=1) | np.all(V_water_data == 0, axis=1))

# Apply mask to U, V, and coords
U_water_data = U_water_data[nonzero_mask]
V_water_data = V_water_data[nonzero_mask]
coords = [c for i, c in enumerate(coords) if nonzero_mask[i]]

print("Filtered Water-only data shape:", U_water_data.shape)
print("Filtered Coordinates length:", len(coords))


In [ ]:
U_water_data.shape

In [ ]:
coords[0]

In [ ]:
# Given:
# - U_water_data.shape = (n_water_points, 100)
# - coords = list of (y, x) tuples, len(coords) == n_water_points
# - You want to reinsert U_water_data into a full U_full_reconstructed of shape (100, 504, 555)

# Initialize a new full-size U matrix with zeros
U_reconstructed = np.zeros((100, 504, 555))

# Insert water data into corresponding coordinates
for idx, (y, x) in enumerate(coords):
    U_reconstructed[:, y, x] = U_water_data[idx]

# Now U_reconstructed is fully rebuilt: land points = 0, water points = correctly restored


In [ ]:
U_reconstructed_reshape = U_reconstructed.transpose(1, 2, 0).reshape(504*555, 100)

In [ ]:
U_reconstructed_reshape_0 = U_reconstructed_reshape[:, 0]

In [ ]:
# Proof of reconstructed matrix

# Step 2: Plot U[0, :, :]
U_t_0 = U_reconstructed[0, :, :]

fig, ax = plt.subplots(figsize=(10, 9))

# Display the grid
cax = ax.imshow(U_t_0, 
                origin='lower',  # (0,0) at bottom-left
                cmap='viridis',  # colormap
                extent=[0, U_t_0.shape[1], 0, U_t_0.shape[0]],  # [x_min, x_max, y_min, y_max]
                aspect='equal')  # square pixels

# Add colorbar
fig.colorbar(cax, ax=ax, label='U[0, y, x] value')

# Step 3: Overlay water points
ax.scatter(x_coords, y_coords, 
           color='cyan', 
           s=1,  # small points
           label='Water points')

# Label axes
ax.set_xlabel('x coordinate')
ax.set_ylabel('y coordinate')
ax.set_title('U[0, y, x] at Time = 0 with Water Points')

# Optional: Add legend
ax.legend(loc='upper right')

plt.show()


### Algorithm
 - Flatten the matrix to shape (num_water_rows, time_steps)
 - perform PCA on the time dimension to summarize the variance of each row of water
 - identify 10 random points that are greater than an arbitrary distance apart
 - compute the sparse correlation matrix for these ten points
 - perform collaborative filtering to predict the remaining correlations of the dimensionality reduced flow regions
 - Identify regions that are at least 80% correlated
 - Compute the true correlations of those regions
 - If two regions have high correlation, explain the results otherwise iterate on this algorithm


In [ ]:
U_water_data.shape

In [ ]:
V_water_data.shape

In [ ]:
len(coords)

### Reducing the dimensionality of the U_water_data matrix with respect to time

In [ ]:
U_svd, S_svd, Vt_svd = np.linalg.svd(U_water_data, full_matrices=False)
explained_variance = np.square(S_svd)
explained_variance_ratio = explained_variance / np.sum(explained_variance)
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

In [ ]:
# Choose k to retain 99% of information
threshold = 0.99
k = np.argmax(cumulative_explained_variance >= threshold) + 1

In [ ]:
print(f"Number of components needed to retain {threshold*100}% information: {k}")

In [ ]:
# Reconstruct the matrix using the top-k components
U_k = U_svd[: , :k]
S_k = np.diag(S_svd[:k])
Vt_k = Vt_svd[:k, :]

U_water_data_reconstructed = U_k @ S_k @ Vt_k

In [ ]:
U_water_data_reconstructed.shape

In [ ]:
pca_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.80))
])

U_water_data_reduced = pca_pipeline.fit_transform(U_water_data)
pca_model = pca_pipeline.named_steps['pca']
print("Number of components selected:", pca_model.n_components_)
print("Shape of reduced data:", U_water_data_reduced.shape)
U_water_data_reduced_t_0 = U_water_data_reduced[:, 0]
U_water_data_reduced_t_0.shape

### Identifying Long Range Distances

In [ ]:
# long_range_indices = select_distant_indices(coords=coords, num_points=100, distance_percent=0.8)

In [ ]:
np.save("long_range_distance_indices.npy", long_range_indices)

In [ ]:
long_range_indices = np.load("long_range_distance_indices.npy")

In [ ]:
U_water_data[long_range_indices].shape

In [ ]:
# U_water_data_reduced_t_0[long_range_indices]

In [ ]:
# # Compute correlation matrix between rows
# n_rows = U_water_data[long_range_indices].shape[0]
# correlation_matrix = np.zeros((n_rows, n_rows))

# for i in range(n_rows):
#     for j in range(n_rows):
#         correlation_matrix[i, j], _ = stats.pearsonr(U_water_data[long_range_indices][i], U_water_data[long_range_indices][j])

In [ ]:
# np.save("correlation_matrix.npy", correlation_matrix)

In [ ]:
correlation_matrix = np.load("correlation_matrix.npy")

In [ ]:
# np.save("v_correlation_matrix.npy", v_correlation_matrix)


In [ ]:
v_correlation_matrix = np.load("v_correlation_matrix.npy")

In [ ]:
# # Compute correlation matrix between rows
# n_rows = V_water_data[long_range_indices].shape[0]
# v_correlation_matrix = np.zeros((n_rows, n_rows))

# for i in range(n_rows):
#     for j in range(n_rows):
#         v_correlation_matrix[i, j], _ = stats.pearsonr(V_water_data[long_range_indices][i], V_water_data[long_range_indices][j])

In [ ]:
sns.heatmap(correlation_matrix)

In [ ]:
sns.heatmap(v_correlation_matrix)

### Correlation matrix above a threshold

In [ ]:
correlation_matrix.shape

In [ ]:
v_correlation_matrix.shape

In [ ]:
v_indices = np.argwhere(v_correlation_matrix > 0.95)

In [ ]:
indices = np.argwhere(correlation_matrix > 0.95)

In [ ]:
indices.shape

In [ ]:
v_indices.shape

In [ ]:
v_indices = v_indices[v_indices[:, 0] != v_indices[:, 1]]

In [ ]:
indices = indices[indices[:, 0] != indices[:, 1]]

In [ ]:
v_indices

In [ ]:
indices

In [ ]:
# Sort each pair (smallest first), so (708, 38) becomes (38, 708)
v_sorted_arr = np.sort(v_indices, axis=1)

# Use a set of tuples to remove duplicates
v_unique_pairs = np.unique(v_sorted_arr, axis=0)

print(v_unique_pairs)

In [ ]:
# Sort each pair (smallest first), so (708, 38) becomes (38, 708)
sorted_arr = np.sort(indices, axis=1)

# Use a set of tuples to remove duplicates
unique_pairs = np.unique(sorted_arr, axis=0)

print(unique_pairs)

In [ ]:
unique_pairs[0]

In [ ]:
[long_range_indices[i] for i in unique_pairs[0]]

In [ ]:
[long_range_indices[i] for i in v_unique_pairs[0]]

In [ ]:
v_correlation_values = [v_correlation_matrix[i[0], i[1]] for i in v_unique_pairs]

In [ ]:
correlation_values = [correlation_matrix[i[0], i[1]] for i in unique_pairs]

In [ ]:
correlation_values

In [ ]:
v_indices

In [ ]:
v_correlation_values

In [ ]:
# Identifying the first vertical pair of points with high correlation
index_y_unique_pair_1_pt_0 = [long_range_indices[i] for i in v_unique_pairs[0]][0] // 504
index_x_unique_pair_1_pt_0 = [long_range_indices[i] for i in v_unique_pairs[0]][0] % 555

index_y_unique_pair_1_pt_1 = [long_range_indices[i] for i in v_unique_pairs[0]][1] // 504
index_x_unique_pair_1_pt_1 = [long_range_indices[i] for i in v_unique_pairs[0]][1] % 555


In [ ]:
# Identifying the first horizontal pair of points with high correlation
index_y_unique_pair_0_pt_0 = [long_range_indices[i] for i in unique_pairs[0]][0] // 504
index_x_unique_pair_0_pt_0 = [long_range_indices[i] for i in unique_pairs[0]][0] % 555

index_y_unique_pair_0_pt_1 = [long_range_indices[i] for i in unique_pairs[0]][1] // 504
index_x_unique_pair_0_pt_1 = [long_range_indices[i] for i in unique_pairs[0]][1] % 555


In [ ]:
index_x_unique_pair_0_pt_0

In [ ]:
index_y_unique_pair_0_pt_0

In [ ]:
index_x_unique_pair_0_pt_1

In [ ]:
index_y_unique_pair_0_pt_1

In [ ]:
correlation_values[0]

In [ ]:
v_correlation_values[0]

In [ ]:
def plot_single_frame(timestep):
    u, v = load_flow_data(timestep)
    fig, ax = plt.subplots()
    plot_vector_field(u, v, ax, f"Flow Field at Time Step {timestep}")
    plt.show()

def plot_vector_field(u, v, ax, title=""):
    ax.clear()
    y, x = np.mgrid[0:u.shape[0], 0:u.shape[1]]
    q = ax.quiver(x * GRID_SPACING_KM, y * GRID_SPACING_KM, u, v,
              compute_magnitude(u, v), scale=50, cmap='viridis')
    
    # Add a colorbar to show magnitude scale
    cbar = plt.colorbar(q, ax=ax)
    cbar.set_label("Velocity Magnitude (km/H)")  # Customize unit if known
    ax.set_title(title)
    ax.set_xlabel("Distance X (km)")
    ax.set_ylabel("Distance Y (km)")
    ax.set_aspect('equal')


In [ ]:

# Plot the land over the water at time 0 

# Assume U and mask are already loaded
# U.shape = (100, 504, 555)
# mask.shape = (504, 555)

MAG_t_0 = MAG[0, :, :]  # time = 0

fig, ax = plt.subplots(figsize=(10, 9))

# Step 1: Display the U field
cax = ax.imshow(U_t_0,
                origin='lower',  # (0,0) bottom-left
                cmap='viridis',
                extent=[0, U_t_0.shape[1], 0, U_t_0.shape[0]],
                aspect='equal')

# Step 2: Overlay the mask
# Create a transparent mask: land is semi-transparent, water is fully transparent
mask_display = np.ma.masked_where(mask == 1, mask)  # mask out water, show land
ax.imshow(mask_display,
          origin='lower',
          cmap='gray',    # land will appear gray
          alpha=0.3,      # transparency (0=transparent, 1=opaque)
          extent=[0, U_t_0.shape[1], 0, U_t_0.shape[0]],
          aspect='equal')

# Plotting the correlated horizontal component
plt.scatter(index_x_unique_pair_0_pt_0, index_y_unique_pair_0_pt_0, marker='x', color='red', label=f"U component correlation: {correlation_values[0]:3f}")
plt.scatter(index_x_unique_pair_0_pt_1, index_y_unique_pair_0_pt_1, marker='x', color='red')

# Plotting the correlated vertical component
plt.scatter(index_x_unique_pair_1_pt_0, index_y_unique_pair_1_pt_0, marker='o', color='orange', label=f"V component correlation: {v_correlation_values[0]:3f}")
plt.scatter(index_x_unique_pair_1_pt_1, index_y_unique_pair_1_pt_1, marker='o', color='orange')
# plt.scatter(index_y_unique_pair_0_pt_1, index_x_unique_pair_0_pt_1, marker='o', color='red')
plt.legend()
# Step 3: Add colorbar and labels
fig.colorbar(cax, ax=ax, label='U[0, y, x] value')

ax.set_xlabel('x coordinate')
ax.set_ylabel('y coordinate')
ax.set_title('U[0, y, x] at Time = 0 with Land Mask Overlay')

plt.show()

In [ ]:
# def plot_single_frame(timestep):
# u, v = load_flow_data(timestep)
# def load_flow_data(timestep):
# u_path = os.path.join(DATA_DIR, f"{timestep}u.csv")
# v_path = os.path.join(DATA_DIR, f"{timestep}v.csv")
# u = np.loadtxt(u_path, delimiter=",")
# v = np.loadtxt(v_path, delimiter=",")
# return u, v
U_t0 = U[0, :, :]
V_t0 = V[0, :, :]
fig, ax = plt.subplots()
# plot_vector_field(u, v, ax, f"Flow Field at Time Step {timestep}")
ax.clear()
y, x = np.mgrid[0:U_t0.shape[0], 0:V_t0.shape[1]]
q = ax.quiver(x * GRID_SPACING_KM, y * GRID_SPACING_KM, U_t0, V_t0,
          compute_magnitude(U_t0, V_t0), scale=50, cmap='viridis')

# Create a transparent mask: land is semi-transparent, water is fully transparent
mask_display = np.ma.masked_where(mask == 1, mask)
ax.imshow(mask_display,
          origin='lower',
          cmap='viridis',    # land will appear gray
          alpha=1,      # transparency (0=transparent, 1=opaque)
          extent=[0, U_t_0.shape[1]* GRID_SPACING_KM, 0, U_t_0.shape[0]*GRID_SPACING_KM],
          aspect='equal')

# Plotting the correlated horizontal component
plt.scatter(index_x_unique_pair_0_pt_0* GRID_SPACING_KM, index_y_unique_pair_0_pt_0* GRID_SPACING_KM, marker='x', color='red', label=f"U component correlation: {correlation_values[0]:3f}")
plt.scatter(index_x_unique_pair_0_pt_1* GRID_SPACING_KM, index_y_unique_pair_0_pt_1* GRID_SPACING_KM, marker='x', color='red')

# Plot correlated horizontal component (U)
x0_u = index_x_unique_pair_0_pt_0 * GRID_SPACING_KM
y0_u = index_y_unique_pair_0_pt_0 * GRID_SPACING_KM
x1_u = index_x_unique_pair_0_pt_1 * GRID_SPACING_KM
y1_u = index_y_unique_pair_0_pt_1 * GRID_SPACING_KM

# Annotate U correlation points
ax.annotate(f"({x0_u:.1f}, {y0_u:.1f})", (x0_u, y0_u), textcoords="offset points", xytext=(5,5), ha='left', fontsize=8, color='red')
ax.annotate(f"({x1_u:.1f}, {y1_u:.1f})", (x1_u, y1_u), textcoords="offset points", xytext=(5,5), ha='left', fontsize=8, color='red')

# Plot correlated vertical component (V)
x0_v = index_x_unique_pair_1_pt_0 * GRID_SPACING_KM
y0_v = index_y_unique_pair_1_pt_0 * GRID_SPACING_KM
x1_v = index_x_unique_pair_1_pt_1 * GRID_SPACING_KM
y1_v = index_y_unique_pair_1_pt_1 * GRID_SPACING_KM

# Plotting the correlated vertical component
plt.scatter(index_x_unique_pair_1_pt_0* GRID_SPACING_KM, index_y_unique_pair_1_pt_0* GRID_SPACING_KM, marker='o', color='orange', label=f"V component correlation: {v_correlation_values[0]:3f}")
plt.scatter(index_x_unique_pair_1_pt_1* GRID_SPACING_KM, index_y_unique_pair_1_pt_1* GRID_SPACING_KM, marker='o', color='orange')

# Annotate V correlation points
ax.annotate(f"({x0_v:.1f}, {y0_v:.1f})", (x0_v, y0_v), textcoords="offset points", xytext=(-50,5), ha='left', fontsize=8, color='red')
# ax.annotate(f"({x1_v:.1f}, {y1_v:.1f})", (x1_v, y1_v), textcoords="offset points", xytext=(-15,15), ha='left', fontsize=8, color='red')


plt.legend()

title = f"Magnitude of Flows at Timestep 0:\nCorrelated Points of Horizontal and Vertical Components"
# Add a colorbar to show magnitude scale
cbar = plt.colorbar(q, ax=ax)
cbar.set_label("Velocity Magnitude (km/H)")  # Customize unit if known
ax.set_title(title)
ax.set_xlabel("Distance X (km)")
ax.set_ylabel("Distance Y (km)")
ax.set_aspect('equal')
plt.show()

In [ ]:
index_x_unique_pair_0_pt_0

In [ ]:
index_y_unique_pair_0_pt_0

In [ ]:
index_x_unique_pair_0_pt_1

In [ ]:
index_y_unique_pair_0_pt_1


In [ ]:
print(f"First pair of correlated U-component flows:\nCorrelation: {correlation_values[0]:3f}\nPoint 1: x={index_y_unique_pair_0_pt_1* GRID_SPACING_KM} km, y={index_x_unique_pair_0_pt_1* GRID_SPACING_KM} km")
print(f"Point 2: x={index_y_unique_pair_0_pt_0* GRID_SPACING_KM} km, y={index_x_unique_pair_0_pt_0* GRID_SPACING_KM} km")

print(f"\nSecond pair of correlated V-component flows:\nCorrelation: {v_correlation_values[0]:3f}\nPoint 1 x={index_y_unique_pair_1_pt_1* GRID_SPACING_KM} km, y={index_x_unique_pair_1_pt_1* GRID_SPACING_KM} km")
print(f"Point 2 x={index_y_unique_pair_1_pt_0* GRID_SPACING_KM} km, y={index_x_unique_pair_1_pt_0* GRID_SPACING_KM} km")

In [ ]:
p1 = (index_y_unique_pair_0_pt_1* GRID_SPACING_KM, index_x_unique_pair_0_pt_1* GRID_SPACING_KM)
p2 = (index_y_unique_pair_0_pt_0* GRID_SPACING_KM, index_x_unique_pair_0_pt_0* GRID_SPACING_KM)
euclidean_distance(p1, p2)

In [ ]:
euclidean_distance(p1, p2)

In [ ]:
p1 = (index_y_unique_pair_1_pt_1* GRID_SPACING_KM, index_x_unique_pair_1_pt_1* GRID_SPACING_KM)
p2 = (index_y_unique_pair_1_pt_0* GRID_SPACING_KM, index_x_unique_pair_1_pt_0* GRID_SPACING_KM)
euclidean_distance(p1, p2)

In [ ]:
def euclidean_distance(p1, p2):
    """Calculate Euclidean distance between two (y, x) points."""
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

### Creating a sparse data matrix

In [ ]:
# Creating a sparse matrix using the identified correlations; to be used for predictions
# I have a matrix of shape (200, 200) and I want to create a matrix of (231705, 231705) based upon this sparse matrix of values. Each position of the correlation matrix corresponds to a larger matrix that is of shape (231705, 100) by using a list object of "long_range_indices".  I want to fill a sparse matrix of shape (231705, 231705) with the coordinates of the "long_range_indices" object.

In [ ]:
# Initialize an empty COO sparse matrix
rows = []
cols = []
data = []

# Fill rows, cols, data
for i in range(1200):
    for j in range(1200):
        corr_value = correlation_matrix[i, j]
        mapped_i = long_range_indices[i]
        mapped_j = long_range_indices[j]
        
        rows.append(mapped_i)
        cols.append(mapped_j)
        data.append(corr_value)

In [ ]:
len(data)

In [ ]:
# np.zeros(shape=(231705, 231705))

In [ ]:
# Create the sparse matrix
sparse_corr_matrix = sp.coo_matrix((data, (rows, cols)), shape=(231705, 231705))

# (Optional) Convert to CSR format for faster matrix operations later
sparse_corr_matrix = sparse_corr_matrix.tocsr()

print(f"Sparse matrix shape: {sparse_corr_matrix.shape}")
print(f"Nonzero elements: {sparse_corr_matrix.nnz}")

In [ ]:
np.save("correlation_matrix.npy", correlation_matrix)

In [ ]:
np.save("long_range_distance_indices.npy", long_range_indices)

## Simulating particle trajectories

In [ ]:
MAG.shape

Algorithm
1. Draw particle directories uniformly at random across the entire map
2. Simulate the particle trajectories for 300 hours and provide:
    - a plot of the initial state
    - a plot of the final state
    - Two plots of intermediary states
3. Distinguish the points with a variety of colors 

In [ ]:
# Draw particle directories uniformly at random across the entire map
x_particle_coordinates = torch.tensor([random.randint(0, MAG[0, :, :].shape[1] -1 ) for _ in range(10)]).reshape(-1, 1) * GRID_SPACING_KM
y_particle_coordinates = torch.tensor([random.randint(0, MAG[0, :, :].shape[0] -1 ) for _ in range(10)]).reshape(-1, 1) * GRID_SPACING_KM

n_particles = y_particle_coordinates.shape[0]
for t in range(1, 100):
    y_new = (y_particle_coordinates[:, t-1] + V[t - 1, (y_particle_coordinates[:, t-1] / 3).int(), (x_particle_coordinates[:, t-1] / 3).int()]) # First hour of movement
    x_new = (x_particle_coordinates[:, t-1] + U[t - 1, (y_particle_coordinates[:, t-1] / 3).int(), (x_particle_coordinates[:, t-1] / 3).int()])

    y_new = (y_new + V[t - 1, (y_new / 3).int(), (x_new / 3).int()]) # Second hour of movement
    x_new = (x_new + U[t - 1, (y_new / 3).int(), (x_new / 3).int()])

    y_new = (y_new + V[t - 1, (y_new / 3).int(), (x_new / 3).int()])
    x_new = (x_new + U[t - 1, (y_new / 3).int(), (x_new / 3).int()])

    y_new = y_new.reshape(n_particles, 1) # Third hour of movement
    x_new = x_new.reshape(n_particles, 1)

    y_particle_coordinates = torch.cat([y_particle_coordinates, y_new], axis=1)
    x_particle_coordinates = torch.cat([x_particle_coordinates, x_new], axis=1)

print(f"x_particle_coordinates.shape: {x_particle_coordinates.shape}")

plot_particle_coordinates_traces_and_trajectories_for_timestep(x_particle_coordinates, y_particle_coordinates, MAG, U, V, TIMESTEP=0, ARROW_SCALE=100)
plot_particle_coordinates_traces_and_trajectories_for_timestep(x_particle_coordinates, y_particle_coordinates, MAG, U, V, TIMESTEP=16, ARROW_SCALE=100)
plot_particle_coordinates_traces_and_trajectories_for_timestep(x_particle_coordinates, y_particle_coordinates, MAG, U, V, TIMESTEP=40, ARROW_SCALE=100)
plot_particle_coordinates_traces_and_trajectories_for_timestep(x_particle_coordinates, y_particle_coordinates, MAG, U, V, TIMESTEP=99, ARROW_SCALE=100)

## Creating a distribution of trajectories from a specific point

In [ ]:
# %%timeit

# Create unique colors for each particle
num_particles = 10

x_index = 100
y_index = 350

# Mean positions for each particle (could also be different)
mean_x = 100
mean_y = 350

# Different variances for each particle
variance_x = np.random.uniform(0, 6, size=num_particles)
variance_y = np.random.uniform(0, 4, size=num_particles)

# Standard deviations are sqrt(variance)
std_x = np.sqrt(variance_x)
std_y = np.sqrt(variance_y)

# Sample particle positions
sampled_x = np.random.normal(loc=mean_x, scale=std_x)
sampled_y = np.random.normal(loc=mean_y, scale=std_y)

# Object Structure: [Particle] [Time]
sampled_y = sampled_y.astype(np.float64).reshape(-1, 1) * GRID_SPACING_KM
sampled_x = sampled_x.astype(np.float64).reshape(-1, 1) * GRID_SPACING_KM
sampled_y = torch.from_numpy(sampled_y).to('cuda')
sampled_x = torch.from_numpy(sampled_x).to('cuda')


n_particles = sampled_y.shape[0]

for t in range(1, 100):
    new_sample_y = sampled_y[:, t-1] + V[t - 1, (sampled_y[:, t-1] / 3).int(), (sampled_x[:, t-1] / 3).int()] # First Hour
    new_sample_x = sampled_x[:, t-1] + U[t - 1, (sampled_y[:, t-1] / 3).int(), (sampled_x[:, t-1] / 3).int()]

    new_sample_y = new_sample_y + V[t - 1, (new_sample_y / 3).int(), (new_sample_x / 3).int()] # Second Hour
    new_sample_x = new_sample_x + U[t - 1, (new_sample_y / 3).int(), (new_sample_x / 3).int()] # Second Hour

    new_sample_y = new_sample_y + V[t - 1, (new_sample_y / 3).int(), (new_sample_x / 3).int()] # Third Hour
    new_sample_x = new_sample_x + U[t - 1, (new_sample_y / 3).int(), (new_sample_x / 3).int()] # Third Hour

    new_sample_y = new_sample_y.reshape(n_particles, 1)
    new_sample_x = new_sample_x.reshape(n_particles, 1)

    sampled_y = torch.cat([sampled_y, new_sample_y], axis=1)
    sampled_x = torch.cat([sampled_x, new_sample_x], axis=1)

# Plotting
plot_particle_debris_movement(sampled_x, sampled_y, MAG, U, V, TIMESTEP=0, ARROW_SCALE=10)
plot_particle_debris_movement(sampled_x, sampled_y, MAG, U, V, TIMESTEP=16, ARROW_SCALE=10)
plot_particle_debris_movement(sampled_x, sampled_y, MAG, U, V, TIMESTEP=24, ARROW_SCALE=10)
plot_particle_debris_movement(sampled_x, sampled_y, MAG, U, V, TIMESTEP=40, ARROW_SCALE=10)

# Problem Set 5

## Using a Kernel Function to predict movement of a particle

In [ ]:
MAG_CPU = MAG.cpu()
TIMESTEP = 0
ARROW_SCALE = 20

fig, ax = plt.subplots(figsize=(12,8))

# Display the grid
cax = ax.imshow(MAG_CPU[TIMESTEP, :, :],
                origin='lower', # (0, 0) at the bottom left 
                cmap = 'viridis', # colormap
                extent=[0, MAG_CPU[TIMESTEP, :, :].shape[1] * GRID_SPACING_KM, 0, MAG_CPU[TIMESTEP, :, :].shape[0] * GRID_SPACING_KM], #[x_min, x_max, y_min, y_max]
                aspect='equal') # square pixel aspect ratio

ax.axhline(y=735, color='red')
ax.axvline(x=105, color='red')

# Add a colorbar
fig.colorbar(cax, ax=ax, label='Magnitude of Flow (Km/H)')

# Label axes
ax.set_xlabel('X coordinate (Km)')
ax.set_ylabel('Y coordinate (Km)')
ax.set_title(f"Identifying an interesting location \nto describe the data \nwith a kernel function \n at Hour {(TIMESTEP) * 3}")

plt.show()

## Estimating the kernel function

In [ ]:
# Each timestep is 72 hours
print(f"Total time in hours: {300 * 24}")
print(f"index of 360 hours: {360 / 72}")

# Coordinates in km
x_point_init = 105
y_point_init = 735

# Coordinates in indices
x_coordinate = torch.tensor(x_point_init // 3)
y_coordinate = torch.tensor(y_point_init // 3)

# Coodinates
print(f"x_coordinate, y_coordinate: {x_coordinate, y_coordinate}")

# Kernel Parameters:
variance = np.array([0.1, 1, 2, 3, 4, 5, 6])
sigma_l = np.sqrt(variance)

tau = 0.1

# 7.2 hours, 36 hours, 72 hours, 144 hours, 288 hours
# 7.2 hours, 1.5 days, 3 days, 6 days, 12 days
ell = np.array([0.1, 0.5, 1, 2, 4])

(
 u_best_sigma, 
 u_best_lengthscale, 
 u_best_log_likelihood, 
 v_best_sigma, 
 v_best_lengthscale, 
 v_best_log_likelihood
) = compute_ll_for_var_lengthscale_of_U_and_V_component(U, V, sigma_l, ell, tau, x_coordinate, y_coordinate, verbose_tau=False)

print(f"u_best_sigma: {u_best_sigma}")
print(f"u_best_lengthscale: {u_best_lengthscale}")
print(f"u_best_log_likelihood: {u_best_log_likelihood}")

print(f"v_best_sigma: {v_best_sigma}")
print(f"v_best_lengthscale: {v_best_lengthscale}")
print(f"v_best_log_likelihood: {v_best_log_likelihood}")


## Running the Squared Exponential Kernel Function for Three More Points

In [ ]:
x_point_1 = 775
y_point_1 = 50

x_point_2 = 415
y_point_2 = 360

x_point_3 = 1075
y_point_3 = 420

TIMESTEP = 0
ARROW_SCALE = 20

fig, ax = plt.subplots(figsize=(12,8))

# Display the grid
cax = ax.imshow(MAG_CPU[TIMESTEP, :, :],
                origin='lower', # (0, 0) at the bottom left 
                cmap = 'viridis', # colormap
                extent=[0, MAG_CPU[TIMESTEP, :, :].shape[1] * GRID_SPACING_KM, 0, MAG_CPU[TIMESTEP, :, :].shape[0] * GRID_SPACING_KM], #[x_min, x_max, y_min, y_max]
                aspect='equal') # square pixel aspect ratio

plt.scatter(x_point_1, y_point_1, label=f"Point of Interest 1: ({x_point_1}, {y_point_1})", color="red", marker="x")
plt.scatter(x_point_2, y_point_2, label=f"Point of Interest 2: ({x_point_2}, {y_point_2})", color="red", marker="^")
plt.scatter(x_point_3, y_point_3, label=f"Point of Interest 3: ({x_point_3}, {y_point_3})", color="red", marker="o")

# Add a colorbar
fig.colorbar(cax, ax=ax, label='Magnitude of Flow (Km/H)')

# Label axes
ax.set_xlabel('X coordinate (Km)')
ax.set_ylabel('Y coordinate (Km)')
ax.set_title(f"Identifying 3 locations of interest \nto describe the data \nwith a kernel function \n at Hour {(TIMESTEP) * 3}")

# Add legend
ax.legend()

plt.show()

### Squared Exponential Kernel Function: Point of Interest 1

In [ ]:
# Coodinates
x_coordinate = torch.tensor(x_point_1 // 3)
y_coordinate = torch.tensor(y_point_1 // 3)
print(f"x_coordinate, y_coordinate: {x_coordinate, y_coordinate}")


# Kernel Parameters:
variance = np.array([0.1, 1, 2, 3, 4, 5, 6])
sigma_l = np.sqrt(variance)

tau = 0.1

# 7.2 hours, 36 hours, 72 hours, 144 hours, 288 hours
# 7.2 hours, 1.5 days, 3 days, 6 days, 12 days
ell = np.array([0.1, 0.5, 1, 2, 4])
(
 u_best_sigma, 
 u_best_lengthscale, 
 u_best_log_likelihood, 
 v_best_sigma, 
 v_best_lengthscale, 
 v_best_log_likelihood
) = compute_ll_for_var_lengthscale_of_U_and_V_component(U, V, sigma_l, ell, tau, x_coordinate, y_coordinate, verbose_tau=False)

print(f"u_best_sigma: {u_best_sigma}")
print(f"u_best_lengthscale: {u_best_lengthscale}")
print(f"u_best_log_likelihood: {u_best_log_likelihood}")

print(f"v_best_sigma: {v_best_sigma}")
print(f"v_best_lengthscale: {v_best_lengthscale}")
print(f"v_best_log_likelihood: {v_best_log_likelihood}")


### Squared Exponential Kernel Function: Point of Interest 2

In [ ]:
# Coodinates
x_coordinate = torch.tensor(x_point_2 // 3)
y_coordinate = torch.tensor(y_point_2 // 3)
print(f"x_coordinate, y_coordinate: {x_coordinate, y_coordinate}")

# Kernel Parameters:
variance = np.array([0.1, 1, 2, 3, 4, 5, 6])
sigma_l = np.sqrt(variance)

tau = 0.1

# 7.2 hours, 36 hours, 72 hours, 144 hours, 288 hours
# 7.2 hours, 1.5 days, 3 days, 6 days, 12 days
ell = np.array([0.1, 0.5, 1, 2, 4])
(
 u_best_sigma, 
 u_best_lengthscale, 
 u_best_log_likelihood, 
 v_best_sigma, 
 v_best_lengthscale, 
 v_best_log_likelihood
) = compute_ll_for_var_lengthscale_of_U_and_V_component(U, V, sigma_l, ell, tau, x_coordinate, y_coordinate, verbose_tau=False)

print(f"u_best_sigma: {u_best_sigma}")
print(f"u_best_lengthscale: {u_best_lengthscale}")
print(f"u_best_log_likelihood: {u_best_log_likelihood}")

print(f"v_best_sigma: {v_best_sigma}")
print(f"v_best_lengthscale: {v_best_lengthscale}")
print(f"v_best_log_likelihood: {v_best_log_likelihood}")


### Squared Exponential Kernel Function: Point of Interest 3

In [ ]:
# Coodinates
x_coordinate = torch.tensor(x_point_3 // 3)
y_coordinate = torch.tensor(y_point_3 // 3)
print(f"x_coordinate, y_coordinate: {x_coordinate, y_coordinate}")

# Kernel Parameters:
variance = np.array([0.1, 1, 2, 3, 4, 5, 6])
sigma_l = np.sqrt(variance)

tau = 0.1

# 7.2 hours, 36 hours, 72 hours, 144 hours, 288 hours
# 7.2 hours, 1.5 days, 3 days, 6 days, 12 days
ell = np.array([0.1, 0.5, 1, 2, 4])
(
 u_best_sigma, 
 u_best_lengthscale, 
 u_best_log_likelihood, 
 v_best_sigma, 
 v_best_lengthscale, 
 v_best_log_likelihood
) = compute_ll_for_var_lengthscale_of_U_and_V_component(U, V, sigma_l, ell, tau, x_coordinate, y_coordinate,  verbose_tau=False)

print(f"u_best_sigma: {u_best_sigma}")
print(f"u_best_lengthscale: {u_best_lengthscale}")
print(f"u_best_log_likelihood: {u_best_log_likelihood}")

print(f"v_best_sigma: {v_best_sigma}")
print(f"v_best_lengthscale: {v_best_lengthscale}")
print(f"v_best_log_likelihood: {v_best_log_likelihood}")


## Varying the Tau Parameter

In [ ]:
# Initial (not for paper)
# Tau is 0.001: Tau is too small
x_coordinate = torch.tensor(105 // 3)
y_coordinate = torch.tensor(735 // 3)

# Kernel Parameters:
variance = np.array([0.1, 1, 2, 3, 4, 5, 6])
sigma_l = np.sqrt(variance)

tau = 0.001

# 7.2 hours, 36 hours, 72 hours, 144 hours, 288 hours
# 7.2 hours, 1.5 days, 3 days, 6 days, 12 days
ell = np.array([0.1, 0.5, 1, 2, 4])

(
 u_best_sigma, 
 u_best_lengthscale, 
 u_best_log_likelihood, 
 v_best_sigma, 
 v_best_lengthscale, 
 v_best_log_likelihood
) = compute_ll_for_var_lengthscale_of_U_and_V_component(U, V, sigma_l, ell, tau, x_coordinate, y_coordinate, verbose_tau=True)

print(f"u_best_sigma: {u_best_sigma}")
print(f"u_best_lengthscale: {u_best_lengthscale}")
print(f"u_best_log_likelihood: {u_best_log_likelihood}")

print(f"v_best_sigma: {v_best_sigma}")
print(f"v_best_lengthscale: {v_best_lengthscale}")
print(f"v_best_log_likelihood: {v_best_log_likelihood}")


In [ ]:
# Tau is 0.01
x_coordinate = torch.tensor(105 // 3)
y_coordinate = torch.tensor(735 // 3)

# Kernel Parameters:
variance = np.array([0.1, 1, 2, 3, 4, 5, 6])
sigma_l = np.sqrt(variance)

tau = 0.01

# 7.2 hours, 36 hours, 72 hours, 144 hours, 288 hours
# 7.2 hours, 1.5 days, 3 days, 6 days, 12 days
ell = np.array([0.1, 0.5, 1, 2, 4])

(
 u_best_sigma, 
 u_best_lengthscale, 
 u_best_log_likelihood, 
 v_best_sigma, 
 v_best_lengthscale, 
 v_best_log_likelihood
) = compute_ll_for_var_lengthscale_of_U_and_V_component(U, V, sigma_l, ell, tau, x_coordinate, y_coordinate, verbose_tau=True)

print(f"u_best_sigma: {u_best_sigma}")
print(f"u_best_lengthscale: {u_best_lengthscale}")
print(f"u_best_log_likelihood: {u_best_log_likelihood}")

print(f"v_best_sigma: {v_best_sigma}")
print(f"v_best_lengthscale: {v_best_lengthscale}")
print(f"v_best_log_likelihood: {v_best_log_likelihood}")


In [ ]:
# Tau is 0.1
x_coordinate = torch.tensor(105 // 3)
y_coordinate = torch.tensor(735 // 3)

# Kernel Parameters:
variance = np.array([0.1, 1, 2, 3, 4, 5, 6])
sigma_l = np.sqrt(variance)

tau = 0.1

# 7.2 hours, 36 hours, 72 hours, 144 hours, 288 hours
# 7.2 hours, 1.5 days, 3 days, 6 days, 12 days
ell = np.array([0.1, 0.5, 1, 2, 4])

(
 u_best_sigma, 
 u_best_lengthscale, 
 u_best_log_likelihood, 
 v_best_sigma, 
 v_best_lengthscale, 
 v_best_log_likelihood
) = compute_ll_for_var_lengthscale_of_U_and_V_component(U, V, sigma_l, ell, tau, x_coordinate, y_coordinate, verbose_tau=True)

print(f"u_best_sigma: {u_best_sigma}")
print(f"u_best_lengthscale: {u_best_lengthscale}")
print(f"u_best_log_likelihood: {u_best_log_likelihood}")

print(f"v_best_sigma: {v_best_sigma}")
print(f"v_best_lengthscale: {v_best_lengthscale}")
print(f"v_best_log_likelihood: {v_best_log_likelihood}")


In [ ]:
# Tau is 1.0
x_coordinate = torch.tensor(105 // 3)
y_coordinate = torch.tensor(735 // 3)

# Kernel Parameters:
variance = np.array([0.1, 1, 2, 3, 4, 5, 6])
sigma_l = np.sqrt(variance)

tau = 1.0

# 7.2 hours, 36 hours, 72 hours, 144 hours, 288 hours
# 7.2 hours, 1.5 days, 3 days, 6 days, 12 days
ell = np.array([0.1, 0.5, 1, 2, 4])

(
 u_best_sigma, 
 u_best_lengthscale, 
 u_best_log_likelihood, 
 v_best_sigma, 
 v_best_lengthscale, 
 v_best_log_likelihood
) = compute_ll_for_var_lengthscale_of_U_and_V_component(U, V, sigma_l, ell, tau, x_coordinate, y_coordinate, verbose_tau=True)

print(f"u_best_sigma: {u_best_sigma}")
print(f"u_best_lengthscale: {u_best_lengthscale}")
print(f"u_best_log_likelihood: {u_best_log_likelihood}")

print(f"v_best_sigma: {v_best_sigma}")
print(f"v_best_lengthscale: {v_best_lengthscale}")
print(f"v_best_log_likelihood: {v_best_log_likelihood}")


## Comparing against known standards for software

In [ ]:
x_point_1 = torch.tensor(105)
y_point_1 = torch.tensor(735)

# Coodinates
x_coordinate = torch.tensor(x_point_1 // 3)
y_coordinate = torch.tensor(y_point_1 // 3)
print(f"x_coordinate, y_coordinate: {x_coordinate, y_coordinate}")

# Kernel Parameters:
variance = np.array([0.1, 1, 2, 3, 4, 5, 6])
sigma_l = np.sqrt(variance)

tau = 0.1

# 0.003 hours, 0.03 hours, 0.3, 1.5 hours, 3 hours, 6 hours, 12 hours
# 1.08 seconds, 10.8 seconds, 1.8 minutes, 18 minutes, 90 minutes, 3 hours, 6 hours, 12 hours
ell = np.array([0.0001, 0.001, 0.01, 0.1, 0.5, 1, 2, 4])

# Create data partitions (k-fold-cross validation)
U_partitions = create_partitions(U, y_coordinate, x_coordinate)
V_partitions = create_partitions(V, y_coordinate, x_coordinate)

U_result_l = []
V_result_l = []

U_ll_dict = {}
V_ll_dict = {}

for index in range(10):

    X_train_U = np.array(range(U_partitions[index]['train'].shape[0])).reshape(-1, 1)
    y_train_U = U_partitions[index]['train'].cpu().numpy()
    X_test_U = np.array(range(U_partitions[index]['test'].shape[0])).reshape(-1, 1)
    y_test_U = U_partitions[index]['test'].cpu().numpy()

    X_train_V = np.array(range(V_partitions[index]['train'].shape[0])).reshape(-1, 1)
    y_train_V = V_partitions[index]['train'].cpu().numpy()
    X_test_V = np.array(range(V_partitions[index]['test'].shape[0])).reshape(-1, 1)
    y_test_V = V_partitions[index]['test'].cpu().numpy()

    # Manual Hyperparameter Search
    for std in sigma_l:
        for l in ell:
            kernel = ConstantKernel(std**2) * RBF(length_scale=l)
            U_model = GaussianProcessRegressor(kernel=kernel)
            V_model = GaussianProcessRegressor(kernel=kernel)
            U_model.fit(X_train_U, y_train_U)
            V_model.fit(X_train_V, y_train_V)
            U_ll = U_model.log_marginal_likelihood_value_
            V_ll = V_model.log_marginal_likelihood_value_
            if index == 0:
                U_ll_dict[(std, l)] = U_ll
                V_ll_dict[(std, l)] = V_ll
            else:
                U_ll_dict[(std, l)] += U_ll
                V_ll_dict[(std, l)] += V_ll


# After folds, average them:
U_ll_dict_avg = {k: v / 10 for k, v in U_ll_dict.items()}
V_ll_dict_avg = {k: v / 10 for k, v in V_ll_dict.items()}

# Now select the best
U_best_params = max(U_ll_dict_avg.items(), key=lambda x: x[1])
V_best_params = max(V_ll_dict_avg.items(), key=lambda x: x[1])

In [ ]:
U_best_params

In [ ]:
V_best_params

## Problem 5

In [ ]:
print(f"v_best_sigma: {v_best_sigma}")
print(f"v_best_lengthscale: {v_best_lengthscale}")
print(f"u_best_sigma: {u_best_sigma}")
print(f"u_best_lengthscale: {u_best_lengthscale}")
best_tau = 0.1

In [ ]:
BEST_SIGMA = 0.31622776601683794
BEST_LENGTHSCALE = 2.0
BEST_TAU = 0.1
x_point_init = 105
y_point_init = 735

In [ ]:
# Initializing the data
U_time = U[:, y_point_init // 3, x_point_init // 3]
V_time = V[:, y_point_init // 3, x_point_init // 3]

U_three_days = torch.zeros(300, device=device, dtype=torch.float64)
U_three_days[[index for index in range(0, 300, 3)]] = U_time

V_three_days = torch.zeros(300, device=device)
V_three_days[[index for index in range(0, 300, 3)]] = V_time

# U_training_mean, U_training_variance = compute_conditional_mean_variance_fixed_hyperparams(U, u_best_sigma, u_best_lengthscale, tau=BEST_TAU, x_coord=x_point_init // 3, y_coord=y_point_init // 3)
# V_training_mean, V_training_variance = compute_conditional_mean_variance_fixed_hyperparams(V, v_best_sigma, v_best_lengthscale, tau=BEST_TAU, x_coord=x_point_init // 3, y_coord=y_point_init // 3)

U_X_test = torch.sort(
    torch.cat([
        torch.arange(1, 300, 3, device=device, dtype=torch.float64),
        torch.arange(2, 300, 3, device=device, dtype=torch.float64)
    ])
)[0].reshape(-1, 1)
U_X_train = torch.arange(0, 300, 3, device=device, dtype=torch.float64).reshape(-1, 1)
U_y_train = U_time.reshape(-1, 1)
U_y_test = causal_moving_average(U_y_train, 5)

V_X_test = torch.sort(
    torch.cat([
        torch.arange(1, 300, 3, device=device, dtype=torch.float64),
        torch.arange(2, 300, 3, device=device, dtype=torch.float64)
    ])
)[0].reshape(-1, 1)
V_X_train = torch.arange(0, 300, 3, device=device, dtype=torch.float64).reshape(-1, 1)
V_y_train = V_time.reshape(-1, 1)
V_y_test = causal_moving_average(V_y_train, 5)

# Making Predictions
U_test_mean, U_test_cov = predict_conditional_mean_variance_fixed_hyperparams(U_X_train, U_X_test, U_y_train, sigma=BEST_SIGMA, ell=BEST_LENGTHSCALE, tau=BEST_TAU)
V_test_mean, V_test_cov = predict_conditional_mean_variance_fixed_hyperparams(V_X_train, V_X_test, V_y_train, sigma=BEST_SIGMA, ell=BEST_LENGTHSCALE, tau=BEST_TAU)

U_test_std = (U_test_cov)**0.5
V_test_std = (V_test_cov)**0.5

U_X_test_CPU = U_X_test.to('cpu')
U_test_mean_CPU = U_test_mean.to('cpu')
U_X_train_CPU = U_X_train.to('cpu')
U_y_train_CPU = U_y_train.to('cpu')
U_test_std_CPU = U_test_std.to('cpu')

plt.figure(figsize=(10, 6))
plt.plot(U_X_test_CPU, U_test_mean_CPU, 'b-', label='Predictive mean')
plt.scatter(U_X_train_CPU, U_y_train_CPU, color='blue', label='Training data', zorder=5)
plt.fill_between(
    U_X_test_CPU.flatten(),
    (U_test_mean_CPU.flatten() + (3 * U_test_std_CPU)),
    (U_test_mean_CPU.flatten() - (3 * U_test_std_CPU)),
    color='orange',
    alpha=0.5,
    label='±3σ band (99.7% CI)',
    lw=1
)
plt.legend()
plt.title('Gaussian Process Prediction with ±3σ Uncertainty Band: Horizontal (U) Component')
plt.xlabel('Day')
plt.ylabel('Horizontal Flow (Km/H)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

V_X_test_CPU = V_X_test.to('cpu')
V_test_mean_CPU = V_test_mean.to('cpu')
V_X_train_CPU = V_X_train.to('cpu')
V_y_train_CPU = V_y_train.to('cpu')
V_test_std_CPU = V_test_std.to('cpu')


plt.figure(figsize=(10, 6))
plt.plot(V_X_test_CPU, V_test_mean_CPU, 'b-', label='Predictive mean')
plt.scatter(V_X_train_CPU, V_y_train_CPU, color='blue', label='Training data', zorder=5)
plt.fill_between(
    V_X_test_CPU.flatten(),
    (V_test_mean_CPU.flatten() + (3 * V_test_std_CPU)),
    (V_test_mean_CPU.flatten() - (3 * V_test_std_CPU)),
    color='orange',
    alpha=0.5,
    label='±3σ band (99.7% CI)',
    lw=1
)
plt.legend()
plt.title('Gaussian Process Prediction with ±3σ Uncertainty Band: Vertical (V) Component')
plt.xlabel('Day')
plt.ylabel('Vertical Flow (Km/H)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Problem 6

## Problem 6A

In [ ]:
# Low-altitude Debris Scattering (80 km sigma)
n_particles = 100

# Mean positions for each particle
mean_x = 100
mean_y = 350

# Different variances for each particle
variance_x = np.random.uniform(0, 80, size=n_particles)
variance_y = np.random.uniform(0, 80, size=n_particles)

# Standard deviations are sqrt(variance)
std_x = np.sqrt(variance_x)
std_y = np.sqrt(variance_y)

# Sample particle positions
sampled_x = np.random.normal(loc=mean_x, scale=std_x)
sampled_y = np.random.normal(loc=mean_y, scale=std_y)

# Object Structure: [Particle] [Time]
sampled_y = sampled_y.astype(np.float64).reshape(-1, 1) * GRID_SPACING_KM
sampled_x = sampled_x.astype(np.float64).reshape(-1, 1) * GRID_SPACING_KM
sampled_y = torch.from_numpy(sampled_y).to('cuda')
sampled_x = torch.from_numpy(sampled_x).to('cuda')

x_particle_coordinates, y_particle_coordinates, U_expanded, V_expanded = simulate_expanded_particle_movement(n_particles=n_particles,U=U,V=V,MAG=MAG, x_particle_coordinates_init=sampled_x, y_particle_coordinates_init=sampled_y, sigma=BEST_SIGMA, lengthscale=BEST_LENGTHSCALE)


In [ ]:
days = [1, 72,]
for day in days:
    plot_particle_coordinates_traces_and_trajectories_for_timestep_days(
        x_particle_coordinates, 
        y_particle_coordinates, 
        MAG, 
        U_expanded, 
        V_expanded, 
        DAY=day,
        show_legend=False)

In [ ]:
plot_particle_coordinates_traces_and_trajectories_for_timestep_days(
        x_particle_coordinates, 
        y_particle_coordinates, 
        MAG, 
        U_expanded, 
        V_expanded, 
        DAY=300, 
        search_locations=[780, 1250, 325, 1300],
        show_legend=True
        )


In [ ]:
MAG_CPU = MAG.cpu()
TIMESTEP = -1
ARROW_SCALE = 20

fig, ax = plt.subplots(figsize=(12,8))

# Display the grid
cax = ax.imshow(MAG_CPU[TIMESTEP, :, :],
                origin='lower', # (0, 0) at the bottom left 
                cmap = 'viridis', # colormap
                extent=[0, MAG_CPU[TIMESTEP, :, :].shape[1] * GRID_SPACING_KM, 0, MAG_CPU[TIMESTEP, :, :].shape[0] * GRID_SPACING_KM], #[x_min, x_max, y_min, y_max]
                aspect='equal') # square pixel aspect ratio

# Original coordinate
ax.axhline(y=1250, color='red')
ax.axvline(x=370, color='red')


# New coordinate
ax.axhline(y=1300, color='green')
ax.axvline(x=325, color='green')


fig.text(
    0.1,
    0.02,
    "* Trajectory vectors are projected by 72 hours for visibility",
    fontsize=10,
    ha="left",
)

# Add a colorbar
fig.colorbar(cax, ax=ax, label='Magnitude of Flow (Km/H)')

# Label axes
ax.set_xlabel('X coordinate (Km)')
ax.set_ylabel('Y coordinate (Km)')
ax.set_title(f"Identifying an interesting location \nto describe the data \nwith a kernel function \n at Hour {(TIMESTEP) * 3}")

plt.show()

In [ ]:
# High-Altitude Mid-Air Debris Scattering (200 km sigma)
n_particles = 100
# Mean positions for each particle
mean_x = 100
mean_y = 350

# Different variances for each particle
variance_x = np.random.uniform(0, 200, size=n_particles)
variance_y = np.random.uniform(0, 200, size=n_particles)

# Standard deviations are sqrt(variance)
std_x = np.sqrt(variance_x)
std_y = np.sqrt(variance_y)

# Sample particle positions
sampled_x = np.random.normal(loc=mean_x, scale=std_x)
sampled_y = np.random.normal(loc=mean_y, scale=std_y)

# Object Structure: [Particle] [Time]
sampled_y = sampled_y.astype(np.float64).reshape(-1, 1) * GRID_SPACING_KM
sampled_x = sampled_x.astype(np.float64).reshape(-1, 1) * GRID_SPACING_KM
sampled_y = torch.from_numpy(sampled_y).to('cuda')
sampled_x = torch.from_numpy(sampled_x).to('cuda')

x_particle_coordinates, y_particle_coordinates, U_expanded, V_expanded = simulate_expanded_particle_movement(n_particles=n_particles,U=U,V=V, MAG=MAG, x_particle_coordinates_init=sampled_x, y_particle_coordinates_init=sampled_y, sigma=BEST_SIGMA, lengthscale=BEST_LENGTHSCALE)


In [ ]:
days = [1, 40, 72, 120]
for day in days:
    plot_particle_coordinates_traces_and_trajectories_for_timestep_days(
        x_particle_coordinates, 
        y_particle_coordinates, 
        MAG, 
        U_expanded, 
        V_expanded, 
        DAY=day)


In [ ]:
plot_particle_coordinates_traces_and_trajectories_for_timestep_days(
        x_particle_coordinates, 
        y_particle_coordinates, 
        MAG, 
        U_expanded, 
        V_expanded, 
        DAY=300, 
        search_locations=[780, 1250, 325, 1300],
        show_legend=True
        )


In [ ]:

# x_particle_coordinates, y_particle_coordinates, U_expanded, V_expanded = simulate_expanded_particle_movement(10, U, V, MAG)


In [ ]:
# torch.save(x_particle_coordinates, 'x_particle_coordinates.pt')
# torch.save(y_particle_coordinates, 'y_particle_coordinates.pt')
# torch.save(U_expanded, "U_expanded.pt")
# torch.save(V_expanded, "V_expanded.pt")


In [ ]:
x_particle_coordinates = torch.load('x_particle_coordinates.pt')
y_particle_coordinates = torch.load('y_particle_coordinates.pt')
U_expanded = torch.load("U_expanded.pt")
V_expanded = torch.load("V_expanded.pt")

In [ ]:
days = [1, 40, 72, 120, 300]
for day in days:
    plot_particle_coordinates_traces_and_trajectories_for_timestep_days(
        x_particle_coordinates, 
        y_particle_coordinates, 
        MAG, 
        U_expanded, 
        V_expanded, 
        DAY=day)


In [ ]:
# import importlib
# import ocean_flow_utility
# importlib.reload(ocean_flow_utility)
# from ocean_flow_utility import *

## Problem 6B

In [3]:
BEST_SIGMA = 0.31622776601683794
BEST_LENGTHSCALE = 2.0
BEST_TAU = 0.1
x_point_init = 105
y_point_init = 735

In [4]:
n_particles = 200

In [ ]:
x_particle_coordinates, y_particle_coordinates, U_expanded, V_expanded = simulate_expanded_particle_debris_scattering(
    U, 
    V, 
    MAG, 
    n_particles=n_particles, 
    sigma=BEST_SIGMA, 
    lengthscale=BEST_LENGTHSCALE
    )

In [ ]:
# torch.save(x_particle_coordinates, 'x_particle_coordinates_0.pt')
# torch.save(y_particle_coordinates, 'y_particle_coordinates_0.pt')
# torch.save(U_expanded, "U_expanded_0.pt")
# torch.save(V_expanded, "V_expanded_0.pt")



In [ ]:
# np.save('x_particle_coordinates.npy', x_particle_coordinates.cpu().numpy())
# np.save('y_particle_coordinates.npy', y_particle_coordinates.cpu().numpy())
# np.save("U_expanded.npy", U_expanded.cpu().numpy())
# np.save("V_expanded.npy", V_expanded.cpu().numpy())

In [5]:
x_particle_coordinates = np.load('x_particle_coordinates.npy')
y_particle_coordinates = np.load('y_particle_coordinates.npy')
U_expanded = np.load("U_expanded.npy")
V_expanded = np.load("V_expanded.npy")

In [ ]:
mask = pd.read_csv("./OceanFlow/mask.csv").to_numpy()
# mask is upside-down
mask = np.flipud(mask)
# Assuming `mask` has shape (503, 555) and `coord_plane` has shape (504, 555)
# Pad the mask at the bottom by 1 row (with zeros)
mask = np.pad(mask, ((0, 1), (0, 0)), mode="constant", constant_values=0)

# # Verify mask
# import matplotlib.pyplot as plt
# # 
# plt.imshow(mask, origin='lower', cmap='gray')  # origin='lower' puts (0,0) bottom-left
# plt.title("Land Mask (0 = land, 1 = ocean)")
# plt.xlabel("x index")
# plt.ylabel("y index")
# plt.colorbar(label="Mask Value")
# plt.show()

# x = np.int64(x_particle_coordinates.cpu().numpy() // GRID_SPACING_KM)
# y = np.int64(y_particle_coordinates.cpu().numpy() // GRID_SPACING_KM)
# for t in range(300):
#     land_mask_t = mask[y[:, 0], x[:, 0]] == 0

#     # Apply the mask to eliminate coordinates over land
#     x_masked = np.where(land_mask_t, np.nan, x)
#     y_masked = np.where(land_mask_t, np.nan, y)


In [6]:
X_imputed_masked_torch_km, y_imputed_masked_torch_km, U_expanded_torch, V_expanded_torch = remove_land_coordinates_and_beach_debris_X_imputed_y_imputed(x_particle_coordinates, y_particle_coordinates, U_expanded, V_expanded)

# Convert Expanded arrays to tensors
U_expanded_torch = torch.tensor(U_expanded, device=device, dtype=torch.float64)
V_expanded_torch = torch.tensor(V_expanded, device=device, dtype=torch.float64)

In [ ]:
# x_masked_torch = torch.tensor(x_masked,device=device, dtype=torch.float64)
# y_masked_torch = torch.tensor(y_masked,device=device, dtype=torch.float64)

In [ ]:
# x_masked_torch = x_masked_torch * GRID_SPACING_KM
# y_masked_torch = y_masked_torch * GRID_SPACING_KM

In [ ]:
# x_particle_coordinates = torch.load('x_particle_coordinates_0.pt')
# y_particle_coordinates = torch.load('y_particle_coordinates_0.pt')
# U_expanded = torch.load("U_expanded_0.pt")
# V_expanded = torch.load("V_expanded_0.pt")

In [ ]:
# Create GIF

plot_n = np.arange(1, 300)
for day in plot_n:
    plot_filename = f"simulation_{day}.png"
    plot_particle_coordinates_traces_and_trajectories_for_timestep_days(X_imputed_masked_torch_km, y_imputed_masked_torch_km, MAG, U_expanded_torch, V_expanded_torch, DAY=day, ARROW_SCALE=72, show_legend=False, save_figure_filename=plot_filename)
plot_filename = f"simulation_{300}.png"
stations = [440, 320, 670, 230, 1100, 680]
plot_particle_coordinates_traces_and_trajectories_for_timestep_days_monitoring_stations(X_imputed_masked_torch_km, y_imputed_masked_torch_km, MAG, U_expanded_torch, V_expanded_torch, DAY=300, ARROW_SCALE=72, stations=stations, save_figure_filename=plot_filename)

# Natural sort using numerical values extracted from filenames
def natural_sort_key(filename):
    match = re.search(r"(\d+)", filename)
    return int(match.group(1)) if match else float("inf")

# Path to your PNG files
output_gif = "output.gif"
# List and sort PNG files
image_folder = "plots"
image_files = [f for f in os.listdir(image_folder) if f.endswith(".png")]
image_files.sort(key=natural_sort_key)
images = []

for file_name in image_files:
    file_path = os.path.join(image_folder, file_name)
    img = Image.open(file_path)
    images.append(img)

# Save to GIF
images[0].save(
    output_gif,
    save_all=True,
    append_images=images[1:],
    duration=100,  # time per frame in milliseconds
    loop=0
)

In [ ]:
days = [1]
for day in days:
    plot_particle_coordinates_traces_and_trajectories_for_timestep_days(X_imputed_masked_torch_km, y_imputed_masked_torch_km, MAG, U_expanded_torch, V_expanded_torch, DAY=day, ARROW_SCALE=72, show_legend=False)


In [ ]:
stations = [440, 320, 670, 230, 1100, 680]
plot_particle_coordinates_traces_and_trajectories_for_timestep_days_monitoring_stations(X_imputed_masked_torch_km, y_imputed_masked_torch_km, MAG, U_expanded_torch, V_expanded_torch, DAY=300, ARROW_SCALE=72, stations=stations)

## Matrix Completion using EM and Gaussian Collaborative Filtering

In [ ]:
# np.save('X_imputed_1200_300.npy', X_imputed)
# np.save('y_imputed_1200_300.npy', y_imputed)
# np.save('U_expanded_1200_300.npy', U_expanded.cpu().numpy())
# np.save('V_expanded_1200_300.npy', V_expanded.cpu().numpy())
# X_imputed = np.load('X_imputed.npy')
# y_imputed = np.load('y_imputed.npy')
# U_expanded = np.load('U_expanded.npy')
# V_expanded = np.load('V_expanded.npy')
# X_imputed_1200_300 = np.load('X_imputed_1200_300.npy')
# y_imputed_1200_300 = np.load('y_imputed_1200_300.npy')
# U_expanded_1200_300 = np.load('U_expanded_1200_300.npy')
# V_expanded_1200_300 = np.load('V_expanded_1200_300.npy')

In [ ]:
BEST_SIGMA = 0.31622776601683794
BEST_LENGTHSCALE = 2.0
BEST_TAU = 0.1
x_point_init = 105
y_point_init = 735

In [ ]:
X_imputed, y_imputed, U_expanded, V_expanded = simulate_expanded_imputed_particle_debris_scattering(    
    U, 
    V, 
    MAG, 
    n_particles_seed = 200,
    n_particles_impute = 1000, 
    sigma=BEST_SIGMA, 
    lengthscale=BEST_LENGTHSCALE
)

In [ ]:
X_imputed_torch_km, y_imputed_torch_km, U_expanded, V_expanded  = remove_land_coordinates_and_beach_debris_X_imputed_y_imputed(X_imputed, y_imputed, U_expanded, V_expanded, mask=None)

In [ ]:
U_expanded_torch = torch.tensor(U_expanded, device=device, dtype=torch.float64)
V_expanded_torch = torch.tensor(V_expanded, device=device, dtype=torch.float64)

In [ ]:
days = [1, 40, 72, 120]
for day in days:
    plot_particle_coordinates_traces_and_trajectories_for_timestep_days(X_imputed_torch, y_imputed_torch, MAG, U_expanded_torch, V_expanded_torch, DAY=day, ARROW_SCALE=72, show_legend=False)


In [ ]:
stations = [650, 500, 800, 1000, 1100, 680]
plot_particle_coordinates_traces_and_trajectories_for_timestep_days_monitoring_stations(X_imputed_torch, y_imputed_torch, MAG, U_expanded_torch, V_expanded_torch, DAY=300, ARROW_SCALE=72, stations=stations)